# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

**Задача**

Построить модель с максимально большим значением *accuracy*, довести долю правильных ответов до 0.75. Проверить *accuracy* на тестовой выборке.

## Обзор данных

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [3]:
df = pd.read_csv('users_behavior.csv')

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Для обучения модели нам понадобятся все столбцы: Признаки - все столбцы, кроме `is_ultra`, цель - `is_ultra`.
Тип данных float, пропуски в данных не найдены.

## Разделение на выборки

In [4]:
#разделим данные на тестовую и тренировочную выборки, 20% и 80%
display (3214 * 0.8)

train_df = df.loc[:2573]
test_df = df.loc[2573:]


2571.2000000000003

In [5]:
#выделим в тренировочной выборке валидационную

features = train_df.drop('is_ultra', axis=1)
target = train_df['is_ultra']


features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)


In [6]:
#!pip install fast_ml
#from fast_ml.model_development import train_valid_test_split

#X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, target = 'is_ultra', train_size=0.6, valid_size=0.2, test_size=0.2)


## Исследование модели

In [7]:
# обучим дерево решений на тренировочной выборке и исследуем варианты глубины дерева

best_decision_tree_model = None
best_result = 0

for depth in range(1, 11):
    
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    
    if result > best_result:
        best_decision_tree_model = model
        best_result = result

display ('Лучший результат дерева решений:', best_result)
display ('Лучшая модель дерева решений:', best_decision_tree_model)

'Лучший результат дерева решений:'

0.782608695652174

'Лучшая модель дерева решений:'

DecisionTreeClassifier(max_depth=6, random_state=12345)

In [8]:
# обучим случайный лес на тренировочной выборке и изучим лучший результат, меняя гиперпараметры

best_random_forest_model = None
best_result = 0

for depth in range(1, 11):
    for est in range(10, 91, 10):
        for leaf in range(1, 5):    
            model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est, min_samples_leaf=leaf)
            model.fit(features_train, target_train)
            predictions_valid = model.predict(features_valid)
            result = accuracy_score(target_valid, predictions_valid)
            
            if result > best_result:
                best_random_forest_model = model
                best_result = result

display ('Лучший результат случайного леса:', best_result)
display ('Лучшая модель случайного леса:', best_random_forest_model)

'Лучший результат случайного леса:'

0.8012422360248447

'Лучшая модель случайного леса:'

RandomForestClassifier(max_depth=9, n_estimators=60, random_state=12345)

In [7]:
# обучим логистическую регрессию на тренировочной выборке

model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)

predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
display ('Результат логистической регрессии:', result)

'Результат логистической регрессии:'

0.6739130434782609

**Вывод**

Модель с наилучшим результатом - модель случайного леса с глубиной равной 9 и количеством деревьев равным 60. Проверим модель на тестовой выборке.

## Тестирование модели

In [9]:
features_test = test_df.drop('is_ultra', axis=1)
target_test = test_df['is_ultra']

best_random_forest_model.fit(features_train, target_train)
predictions_test = best_random_forest_model.predict(features_test)
result = accuracy_score(target_test, predictions_test)

print ('Точность прогнозирования тестовой выборки:', result)

Точность прогнозирования тестовой выборки: 0.8268330733229329


## Проверка на адекватность

In [10]:
# сравним точность полученной модели с точностью рандомного предсказания 
import random

random_predictions = []
for num in range(1, 642):
    num = random.randint(0, 1)
    random_predictions.append(num)
    
result_random = accuracy_score(target_test, random_predictions)

print ('Точность случайных предсказаний: ', result_random)
print ('Точность предсказаний модели: ', result)

Точность случайных предсказаний:  0.5179407176287052
Точность предсказаний модели:  0.8268330733229329


In [11]:
# сравним точность полученной модели с точностью модели, которая предсказывает наиболее часто встречающийся класс

dummy_model =  DummyClassifier(strategy="most_frequent")
dummy_model.fit(features_train, target_train)
dummy_predictions = dummy_model.predict(features_test)

dummy_result = accuracy_score(target_test, dummy_predictions)
print ('Точность модели dummy: ', dummy_result)
print ('Точность нашей модели: ', result)

Точность модели dummy:  0.6973478939157566
Точность нашей модели:  0.8268330733229329


### Вывод
Наиболее удачная обученная модель - модель случайного леса с глубиной равной 9 и количеством деревьев равным 60. Данная модель прошла тестирование на рандомной выборке и также показала результат выше, чем модель, которая предсказывает наиболее часто встречающийся класс. Данная модель дает точность выше 0.8 и поможет предсказать наиболее подходящий для клиента тариф.